<a href="https://colab.research.google.com/github/yonathanarbel/AI-LAW/blob/main/Class_3_in_Class_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google.generativeai as genai
import numpy as np

# Configure the API key
API_KEY = "AIzaSyAmGmHuMeKN-HNKor7TItl4uSlDeHLZAd"

genai.configure(api_key=API_KEY)
# Create an embedding model
model = "models/embedding-001"


# Create a list of strings
strings = [
    "King",
    "Queen",
    "Man",
    "Woman"
]



# Generate embeddings for each string
embeddings = []
for string in strings:
    embedding = genai.embed_content(
        model=model,
        content=string,
        task_type="retrieval_document"
    )
    embeddings.append(embedding)

# Print the embeddings and their dimensionality
for i, embedding in enumerate(embeddings):
    print(f"Embedding for string {i + 1}:")
    print(embedding)
    print(f"This embedding has {len(embedding['embedding'])} dimensions.")
    print("\n")

# Print the statement about dimensionality
print(f"Each string has {len(embeddings[0]['embedding'])} dimensions. That's quite long!")

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Assuming 'embeddings' is your list of embeddings from the previous cell
# Extract the embedding vectors
embedding_vectors = [emb['embedding'] for emb in embeddings]

# Convert to numpy array
embedding_array = np.array(embedding_vectors)

# Check the number of samples
n_samples = embedding_array.shape[0]

if n_samples < 4:
    # Use PCA for very small datasets
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embedding_array)
    method = "PCA"
else:
    # Use t-SNE with adjusted perplexity
    perplexity = min(30, n_samples - 1)  # Adjust perplexity based on sample size
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    embeddings_2d = tsne.fit_transform(embedding_array)
    method = "t-SNE"

# Plot the 2D embeddings
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

# Label each point
for i, txt in enumerate(strings):
    plt.annotate(txt, (embeddings_2d[i, 0], embeddings_2d[i, 1]))

plt.title(f"2D Representation of Embeddings using {method}")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate pairwise distances
distances = np.linalg.norm(embeddings_2d[:, None] - embeddings_2d, axis=2)

# Plot vectors and distances
plt.figure(figsize=(12, 10))

# Draw vectors
for i, (x, y) in enumerate(embeddings_2d):
    plt.quiver(0, 0, x, y, angles='xy', scale_units='xy', scale=1, label=strings[i])

# Draw distances
for i in range(len(embeddings_2d)):
    for j in range(i+1, len(embeddings_2d)):
        x1, y1 = embeddings_2d[i]
        x2, y2 = embeddings_2d[j]
        plt.plot([x1, x2], [y1, y2], 'r--', alpha=0.3)
        midpoint = ((x1+x2)/2, (y1+y2)/2)
        plt.text(midpoint[0], midpoint[1], f"{distances[i,j]:.2f}", fontsize=8)

plt.title("Vectors and Distances between Word Embeddings")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.legend()
plt.axis('equal')
plt.grid(True)
plt.show()

# Print pairwise distances
print("Pairwise Distances:")
for i in range(len(strings)):
    for j in range(i+1, len(strings)):
        print(f"Distance between '{strings[i]}' and '{strings[j]}': {distances[i,j]:.2f}")

# Calculate and print vector operations
print("\nVector Operations:")
king = embedding_vectors[strings.index("King")]
queen = embedding_vectors[strings.index("Queen")]
man = embedding_vectors[strings.index("Man")]
woman = embedding_vectors[strings.index("Woman")]

result = np.array(queen) - np.array(king) + np.array(man)


In [ ]:

# Get embeddings for "King", "Man", and "Woman"
king_embedding = embeddings[0]['embedding']
man_embedding = embeddings[2]['embedding']
woman_embedding = embeddings[3]['embedding']

# Perform the arithmetic operation: King - Man + Woman
result_embedding = np.array(king_embedding) - np.array(man_embedding) + np.array(woman_embedding)


def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_a = np.linalg.norm(vec1)
    norm_b = np.linalg.norm(vec2)
    return dot_product / (norm_a * norm_b)

# Compare the result_embedding with the available embeddings and print cosine similarity
for i, embedding in enumerate(embeddings):
    similarity = cosine_similarity(result_embedding, embedding['embedding'])


# Compare the result_embedding with the available embeddings and calculate cosine similarity
similarities = []
for i, embedding in enumerate(embeddings):
    similarity = cosine_similarity(result_embedding, embedding['embedding'])
    similarities.append((strings[i], round(similarity, 3)))

# Sort the results by similarity
sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

# Print the sorted results
for string, similarity in sorted_similarities:
    print(f"Cosine similarity between result and string {string}: {similarity}")
